In [1]:
from functions_training_pipeline import Model
import functions_training_pipeline as f

In [2]:
rf = f.load_object('RF_test_2023_04_17')

In [3]:
rf.get_attributes()

model = <class 'sklearn.ensemble._forest.RandomForestRegressor'>
hyperparameters = [{'max_depth': 4, 'n_estimators': 10}, {'max_depth': 4, 'n_estimators': 20}, {'max_depth': 7, 'n_estimators': 10}, {'max_depth': 7, 'n_estimators': 20}]
name = RF_test_2023_04_17
date_tested = 2023-04-17
columns = ['x', 'y', 'mw_value', 'col', 'row']
dates = [numpy.datetime64('2017-08-01T00:00:00.000000000'), numpy.datetime64('2017-08-02T00:00:00.000000000'), numpy.datetime64('2017-08-03T00:00:00.000000000'), numpy.datetime64('2017-08-04T00:00:00.000000000'), numpy.datetime64('2017-08-05T00:00:00.000000000'), numpy.datetime64('2017-08-06T00:00:00.000000000')]
best_hyperparameter_list = [{'max_depth': 4, 'n_estimators': 20}, {'max_depth': 4, 'n_estimators': 20}, {'max_depth': 4, 'n_estimators': 10}, {'max_depth': 4, 'n_estimators': 10}, {'max_depth': 7, 'n_estimators': 20}]
feature_importance_list = [{'x': 0.1438303719158582, 'y': 0.3450341471584463, 'mw_value': 0.12571045277256684, 'col': 0.1550098214779

In [4]:
rf.best_hyperparameter_list

[{'max_depth': 4, 'n_estimators': 20},
 {'max_depth': 4, 'n_estimators': 20},
 {'max_depth': 4, 'n_estimators': 10},
 {'max_depth': 4, 'n_estimators': 10},
 {'max_depth': 7, 'n_estimators': 20}]

In [4]:
rf.get_results()

,Metric,Train,Test
0,RMSE,0.704961,1.469092
1,RMSE_std,0.057279,0.592755
2,R2,0.290228,-0.845890
3,R2_std,0.035167,1.095800


In [4]:
dt = f.load_object('DT_test')

In [5]:
dt.get_attributes()

model = <class 'sklearn.tree._classes.DecisionTreeRegressor'>
hyperparameters = [{'max_depth': 4, 'min_samples_leaf': 2}, {'max_depth': 4, 'min_samples_leaf': 5}, {'max_depth': 7, 'min_samples_leaf': 2}, {'max_depth': 7, 'min_samples_leaf': 5}]
name = DT_test
dates = [numpy.datetime64('2019-07-01T00:00:00.000000000'), numpy.datetime64('2019-07-02T00:00:00.000000000'), numpy.datetime64('2019-07-03T00:00:00.000000000'), numpy.datetime64('2019-07-04T00:00:00.000000000')]
best_hyperparameter_list = [{'max_depth': 4, 'min_samples_leaf': 2}, {'max_depth': 4, 'min_samples_leaf': 2}, {'max_depth': 4, 'min_samples_leaf': 2}, {'max_depth': 4, 'min_samples_leaf': 2}, {'max_depth': 4, 'min_samples_leaf': 2}]
feature_importance_list = [{'x': 0.16273379573715818, 'y': 0.2978891678455866, 'mw_value': 0.0, 'mean_3': 0.5393770364172551}, {'x': 0.5931008062031214, 'y': 0.13366970086237703, 'mw_value': 0.0, 'mean_3': 0.27322949293450155}, {'x': 0.1736382305940472, 'y': 0.34351470359039377, 'mw_value': 0.

In [6]:
dt.get_results()

     Metric     Train      Test
0      RMSE  0.083774  0.127109
1  RMSE_std  0.006762  0.027293
2        R2  0.475871 -0.093088
3    R2_std  0.049463  0.577532


,Metric,Train,Test
0,RMSE,0.083774,0.127109
1,RMSE_std,0.006762,0.027293
2,R2,0.475871,-0.093088
3,R2_std,0.049463,0.577532


In [7]:
rf.get_results()

     Metric     Train      Test
0      RMSE  0.704961  1.469092
1  RMSE_std  0.057279  0.592755
2        R2  0.290228 -0.845890
3    R2_std  0.035167  1.095800


,Metric,Train,Test
0,RMSE,0.704961,1.469092
1,RMSE_std,0.057279,0.592755
2,R2,0.290228,-0.845890
3,R2_std,0.035167,1.095800


In [8]:
import pandas as pd

In [14]:
d = {'': rf.get_results().pop('Metric'), 'rf' : rf.get_results()[['Train', 'Test']], 'dt' : dt.get_results()[['Train', 'Test']]}
pd.concat(d.values(), axis=1, keys=d.keys())


     Metric     Train      Test
0      RMSE  0.704961  1.469092
1  RMSE_std  0.057279  0.592755
2        R2  0.290228 -0.845890
3    R2_std  0.035167  1.095800
     Metric     Train      Test
0      RMSE  0.704961  1.469092
1  RMSE_std  0.057279  0.592755
2        R2  0.290228 -0.845890
3    R2_std  0.035167  1.095800
     Metric     Train      Test
0      RMSE  0.083774  0.127109
1  RMSE_std  0.006762  0.027293
2        R2  0.475871 -0.093088
3    R2_std  0.049463  0.577532


rf                  dt          
     Metric     Train      Test     Train      Test
0      RMSE  0.704961  1.469092  0.083774  0.127109
1  RMSE_std  0.057279  0.592755  0.006762  0.027293
2        R2  0.290228 -0.845890  0.475871 -0.093088
3    R2_std  0.035167  1.095800  0.049463  0.577532